**When we train object detection for custom datasets, we often get confused over how to choose hyperparameters for the Network. Anchor boxes (one of the hyperparameters) are very important to detect objects with different scales and aspect ratios. We will get improved detection results if we get the anchors right.**

In [ ]:
import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import os
from glob import glob
import pandas as pd
from tqdm import tqdm
import time
import multiprocessing
import seaborn as sns
import collections
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [ ]:
def read_xray(path):
    dicom = pydicom.read_file(path)
    width = dicom.Rows
    height = dicom.Columns
    basename = os.path.basename(path).split('.')[0]
    return basename, width, height

**Count CPU**

In [ ]:
cpu_count = multiprocessing.cpu_count()

In [ ]:
train = pd.read_csv('../input/vinbigdata-chest-xray-abnormalities-detection/train.csv')
image_paths = "../input/vinbigdata-chest-xray-abnormalities-detection/train"
image_paths = glob(os.path.join(image_paths, '*.dicom'))

**Load Shape Image with MultiProcessing**

In [ ]:
data = pd.DataFrame()
image_id = []
widths = []
heights = []
start = time.time()
try:
    pool = multiprocessing.Pool(processes = cpu_count)
    for basename, width, height in pool.map(read_xray, image_paths):
        image_id.append(basename)
        widths.append(width)
        heights.append(height)
finally:
    pool.close()
    pool.join()
data['image_id'] = image_id
data['heights'] = heights
data['widths'] = widths
print('Time Execution : {}'.format(time.time() - start))

**Merge data contain shape Image with data origin**

In [ ]:
data = pd.merge(train, data, on='image_id', how='inner')

**Filter Bounding Boxes in Data**

In [ ]:
data = data[data['class_name'] != 'No finding']

**Analysis of bounding boxes (Training data)**

**Image resizing**

In [ ]:
def change_to_wh(data):
    data['w'] = data['x_max'] - data['x_min'] + 1
    data['h'] = data['y_max'] - data['y_min'] + 1
    return data

min_dimension = 600
max_dimension = 1024

def _compute_new_static_size(width, height, min_dimension, max_dimension):
    orig_height = height
    orig_width = width
    orig_min_dim = min(orig_height, orig_width)
  
    # Calculates the larger of the possible sizes
    large_scale_factor = min_dimension / float(orig_min_dim)
    large_height = int(round(orig_height * large_scale_factor))
    large_width = int(round(orig_width * large_scale_factor))
    large_size = [large_height, large_width]
    if max_dimension:
    # Calculates the smaller of the possible sizes, use that if the larger is too big.
        orig_max_dim = max(orig_height, orig_width)
        small_scale_factor = max_dimension / float(orig_max_dim)
        small_height = int(round(orig_height * small_scale_factor))
        small_width = int(round(orig_width * small_scale_factor))
        small_size = [small_height, small_width]
        new_size = large_size
    if max(large_size) > max_dimension:
        new_size = small_size
    else:
        new_size = large_size
    
    return new_size[1], new_size[0]

In [ ]:
data.describe()

**Rescale Boxes**

In [ ]:
data = change_to_wh(data)
data['new_w'], data['new_h'] = np.vectorize(_compute_new_static_size)(data['widths'],  data['heights'], min_dimension, max_dimension)
data['b_w'] = data['new_w'] * data['w']/ data['widths']
data['b_h'] = data['new_h'] * data['h'] / data['heights']
data['b_ar'] = data['b_w'] / data['b_h']

**Distribution of Bounding Boxes!**

In [ ]:
sns.jointplot(x="b_w", y="b_h", data=data)
sns.jointplot(x="b_w", y="b_h", data=data, kind='kde')

**Calculate the base box size!****

In [ ]:

def count_base_size(width, height, input_array=[64, 96, 128, 196, 212, 256, 512]):
    result = {}
    for ele in input_array:
        result[str(ele)] = 0
    result['rest'] = 0
    
    for w, h in zip(width, height):
        done = False
        for inp in input_array:
            if w <= inp and h <= inp:
                result[str(inp)] += 1
                done = True
        if done == False:
            result['rest'] += 1
            
    return result
    
D = count_base_size(data["b_w"].tolist(), data["b_h"].tolist())
OD = collections.OrderedDict(sorted(D.items()))
plt.bar(range(len(OD)), OD.values(), align='center')
plt.xticks(range(len(OD)), OD.keys())

plt.show()

**Cluster bbox (width, height) on eucledian distance metric**


In [ ]:
X = data[['b_w', 'b_h']].values
K = KMeans(5, random_state=0)
labels = K.fit(X)
plt.scatter(X[:, 0], X[:, 1], c=labels.labels_, s=50, cmap='viridis');

In [ ]:
out = labels.cluster_centers_

ar = out[:,0] / out[:,1]
scale = out[:,1] * np.sqrt(ar) / 256

print("Aspect Ratios: {}".format(ar))

print("Scales: {}".format(scale))


**IOU based clusterring**

In [ ]:
import numpy as np

def iou(box, clusters):
    """
    Calculates the Intersection over Union (IoU) between a box and k clusters.
    :param box: tuple or array, shifted to the origin (i. e. width and height)
    :param clusters: numpy array of shape (k, 2) where k is the number of clusters
    :return: numpy array of shape (k, 0) where k is the number of clusters
    """
    x = np.minimum(clusters[:, 0], box[0])
    y = np.minimum(clusters[:, 1], box[1])
    if np.count_nonzero(x == 0) > 0 or np.count_nonzero(y == 0) > 0:
        raise ValueError("Box has no area")

    intersection = x * y
    box_area = box[0] * box[1]
    cluster_area = clusters[:, 0] * clusters[:, 1]

    iou_ = intersection / (box_area + cluster_area - intersection)

    return iou_


def avg_iou(boxes, clusters):
    """
    Calculates the average Intersection over Union (IoU) between a numpy array of boxes and k clusters.
    :param boxes: numpy array of shape (r, 2), where r is the number of rows
    :param clusters: numpy array of shape (k, 2) where k is the number of clusters
    :return: average IoU as a single float
    """
    return np.mean([np.max(iou(boxes[i], clusters)) for i in range(boxes.shape[0])])


def translate_boxes(boxes):
    """
    Translates all the boxes to the origin.
    :param boxes: numpy array of shape (r, 4)
    :return: numpy array of shape (r, 2)
    """
    new_boxes = boxes.copy()
    for row in range(new_boxes.shape[0]):
        new_boxes[row][2] = np.abs(new_boxes[row][2] - new_boxes[row][0])
        new_boxes[row][3] = np.abs(new_boxes[row][3] - new_boxes[row][1])
    return np.delete(new_boxes, [0, 1], axis=1)


def kmeans(boxes, k, dist=np.median):
    """
    Calculates k-means clustering with the Intersection over Union (IoU) metric.
    :param boxes: numpy array of shape (r, 2), where r is the number of rows
    :param k: number of clusters
    :param dist: distance function
    :return: numpy array of shape (k, 2)
    """
    rows = boxes.shape[0]

    distances = np.empty((rows, k))
    last_clusters = np.zeros((rows,))

    np.random.seed()

    # the Forgy method will fail if the whole array contains the same rows
    clusters = boxes[np.random.choice(rows, k, replace=False)]

    while True:
        for row in range(rows):
            distances[row] = 1 - iou(boxes[row], clusters)

        nearest_clusters = np.argmin(distances, axis=1)

        if (last_clusters == nearest_clusters).all():
            break

        for cluster in range(k):
            clusters[cluster] = dist(boxes[nearest_clusters == cluster], axis=0)

        last_clusters = nearest_clusters

    return clusters

In [ ]:
X = data[['b_w', 'b_h']].values
# Cluster with 5 centers
cl = kmeans(X, 5)
print (cl)

In [ ]:
ar_iou = cl[:,0] / cl[:,1]
print(ar_iou)
scale_iou = cl[:,1] * np.sqrt(ar_iou) / 256
print(scale_iou)

# **Don't forget upvote for me :))**